Vamos a conectarnos a la base con datos históricos de baloncesto, con los siguientes parámetros:  
- servidor: relational.fit.cvut.cz
- usuario: guest
- contraseña: relational
- base de datos: Basketball_men

Enlace al diagrama del modelo relacional:  
https://relational.fit.cvut.cz/assets/img/datasets-generated/Basketball_men.svg  

Obtén los jugadores con al menos 30.000 puntos en su carrera, que hayan obtenido algún premio y continúen vivos. Obtener su altura en metros (en la base de datos está en pulgadas).

In [45]:
import pymysql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [46]:
database_host = 'relational.fit.cvut.cz'
username = 'guest'
password = 'relational'
database_name = 'Basketball_men'

db = pymysql.connect(host=database_host,
                     user=username,
                     password=password,
                     database=database_name)

In [47]:
import pandas as pd
df = pd.read_sql('SHOW TABLES',db)
df

,Tables_in_Basketball_men
0,awards_coaches
1,awards_players
2,coaches
3,draft
4,player_allstar
5,players
6,players_teams
7,series_post
8,teams


In [5]:
query = '''
SELECT T1.playerID,
       TotalPointsTeams + COALESCE(TotalPointsAllStar,0) AS TotalPoints,
       firstName, lastName, height*0.0254 as meters
FROM 

    (SELECT playerID,
           SUM(points) as TotalPointsTeams
    FROM players_teams
    GROUP BY playerID)T1

    LEFT JOIN

    (SELECT playerID,
           SUM(points) as TotalPointsAllStar
    FROM player_allstar
    GROUP BY playerID)T2

    ON T1.playerId = T2.playerID
    
    JOIN players
    ON T1.playerId = players.playerID
    

WHERE TotalPointsTeams + COALESCE(TotalPointsAllStar,0) >=30000
AND T1.playerID IN (SELECT DISTINCT playerID from awards_players)
AND deathDate = '0000-00-00'
'''

df = pd.read_sql(query,db)
df

,playerID,TotalPoints,firstName,lastName,meters
0,abdulka01,38638.0,Kareem,Abdul-Jabbar,2.1590
1,ervinju01,30347.0,Julius,Erving,1.9812
2,jordami01,32554.0,Michael,Jordan,1.9812
3,malonka01,37073.0,Karl,Malone,2.0574


Mismo ejercicio con pandas

In [ ]:
players_teams = pd.read_sql('SELECT * FROM players_teams',db)
player_allstar = pd.read_sql('SELECT * FROM player_allstar',db)
players = pd.read_sql('SELECT * FROM players',db)
awards_players = pd.read_sql('SELECT * FROM awards_players',db)

In [20]:
puntos_teams = players_teams[['playerID','points']].groupby('playerID',as_index=False).sum()
puntos_teams.columns=['playerID','TotalPointsTeam']
puntos_teams.head()

,playerID,TotalPointsTeam
0,abdelal01,1465
1,abdulka01,38387
2,abdulma01,9087
3,abdulma02,8553
4,abdulta01,1830


In [21]:
puntos_allstar = player_allstar[['playerID','points']].groupby('playerID',as_index=False).sum()
puntos_allstar.columns=['playerID','TotalPointsAllStar']
puntos_allstar.head()

,playerID,TotalPointsAllStar
0,abdulka01,251.0
1,abdursh01,9.0
2,adamsal01,4.0
3,adamsmi01,9.0
4,aguirma01,36.0


In [26]:
puntos_total = puntos_teams.merge(puntos_allstar,how='left').fillna(0)
puntos_total['TotalPoints'] = puntos_total.TotalPointsTeam + puntos_total.TotalPointsAllStar
puntos_total_30k = puntos_total.loc[puntos_total.TotalPoints>30000]
puntos_total_30k

,playerID,TotalPointsTeam,TotalPointsAllStar,TotalPoints
1,abdulka01,38387,251.0,38638.0
726,chambwi01,31419,191.0,31610.0
1253,ervinju01,30026,321.0,30347.0
2248,jordami01,32292,262.0,32554.0
2695,malonka01,36928,145.0,37073.0


In [30]:
puntos_total_30k_awards = puntos_total_30k.loc[puntos_total_30k['playerID'].isin(awards_players.playerID.unique())]
puntos_total_30k_awards

,playerID,TotalPointsTeam,TotalPointsAllStar,TotalPoints
1,abdulka01,38387,251.0,38638.0
726,chambwi01,31419,191.0,31610.0
1253,ervinju01,30026,321.0,30347.0
2248,jordami01,32292,262.0,32554.0
2695,malonka01,36928,145.0,37073.0


In [37]:
df = puntos_total_30k_awards.merge(players[['playerID','firstName','lastName','height','deathDate']])
df

,playerID,TotalPointsTeam,TotalPointsAllStar,TotalPoints,firstName,lastName,height,deathDate
0,abdulka01,38387,251.0,38638.0,Kareem,Abdul-Jabbar,85.0,0000-00-00
1,chambwi01,31419,191.0,31610.0,Wilton,Chamberlain,85.0,1999-10-12
2,ervinju01,30026,321.0,30347.0,Julius,Erving,78.0,0000-00-00
3,jordami01,32292,262.0,32554.0,Michael,Jordan,78.0,0000-00-00
4,malonka01,36928,145.0,37073.0,Karl,Malone,81.0,0000-00-00


In [39]:
df = df.loc[df.deathDate=='0000-00-00']
df

,playerID,TotalPointsTeam,TotalPointsAllStar,TotalPoints,firstName,lastName,height,deathDate
0,abdulka01,38387,251.0,38638.0,Kareem,Abdul-Jabbar,85.0,0000-00-00
2,ervinju01,30026,321.0,30347.0,Julius,Erving,78.0,0000-00-00
3,jordami01,32292,262.0,32554.0,Michael,Jordan,78.0,0000-00-00
4,malonka01,36928,145.0,37073.0,Karl,Malone,81.0,0000-00-00


In [41]:
df['heightMetros'] = df.height * 0.0254
df[['firstName','lastName','TotalPoints','heightMetros']]

,firstName,lastName,TotalPoints,heightMetros
0,Kareem,Abdul-Jabbar,38638.0,2.1590
2,Julius,Erving,30347.0,1.9812
3,Michael,Jordan,32554.0,1.9812
4,Karl,Malone,37073.0,2.0574
